# IAU 2023/2024
## **Autori:** Laura Fulajtárová (50%), Bruno Kristián (50%)

# Fáza 2 - Predspracovanie údajov

In [484]:
import pandas as pd
import matplotlib
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from scipy.stats import iqr
import statsmodels.stats as sm_stats
from sklearn import preprocessing
import statsmodels.stats.api as sms
import statsmodels.api as sm
from scipy.stats import ttest_ind

In [485]:
product = pd.read_csv("product_phase_1.csv", sep='\t')
user = pd.read_csv("user_phase_1.csv", sep='\t')
session = pd.read_csv("session_phase_1.csv", sep='\t')

### Datasets merge

In [486]:
set1 = set(session['product_ean'])
set2 = set(product['product_ean'])
common_values = set1.intersection(set2)

if common_values:
    print("Common values found in 'product_ean'.")
else:
    print("No common values found in 'product_ean'.")

No common values found in 'product_ean'.


Zistili sme, že datasety nevieme spojiť na základe stĺpca "product_ean", pretože sme nenašli žiadne spoločné hodnoty.  

In [487]:
set1 = set(session['user_id'])
set2 = set(user['old_user_id'])
common_values = set1.intersection(set2)

if common_values:
    print("Common values found in 'user_id'.")
else:
    print("No common values found in 'user_id'.")

Common values found in 'user_id'.


Vedeli by sme spojiť datasety na základe stĺpca "user_id" a "old_user_id". Tieto dva stĺpce sú v oboch datasetoch a majú spoločné hodnoty. 

In [488]:
user.rename(columns={'user_id': 'new_user_id', 'old_user_id': 'user_id'}, inplace=True)

In [489]:
merged_data= pd.merge(user, session, on='user_id', how='inner')

Spojili sme datasety na základe stĺpa "user_id". Použili sme funkciu merge z knižnice pandas s parametrom how = "inner". 

Vznikol nám nový dataset, ktorý obsahuje všetky stĺpce z oboch datasetov.

### Nahradenie chýbajúcich hodnôt

In [490]:
merged_data.isnull().sum()

birthdate                         4929
sex                               5497
registration                         0
new_user_id                          0
race                              7732
user_id                              0
pct_mouse_click                     19
pct_scroll_move                     13
pct_doubleclick                     10
pct_input                            8
session_start                        0
page_activity_duration              14
ack                                  0
pct_click                           17
pct_mouse_move                      10
browser_name                         0
pct_scrandom                         6
pct_scroll_move_duration            10
mouse_move_total_rel_distance       16
product_ean                          0
pct_rage_click                      16
pct_wild_mouse                      20
session_duration                     0
wild_mouse_duration                 14
session_id                           0
pct_click_product_info   

In [491]:
def custom_imputer(column_with_nulls, desired_distribution):
    # Calculate the number of null values to fill
    total_count = len(column_with_nulls)
    num_nulls_to_fill = column_with_nulls.isnull().sum()

    # Calculate the number of occurrences for each race value
    counts = {value: int((percentage / 100) * total_count) for value, percentage in desired_distribution.items()}

    # Generate a list of race values to fill nulls
    list_of_values = []
    for value, count in counts.items():
        list_of_values.extend([value] * count)

    # Randomly shuffle the list to maintain the distribution
    np.random.shuffle(list_of_values)

    # Fill null values with the shuffled race values
    column_with_nulls[column_with_nulls.isnull()] = list_of_values[:num_nulls_to_fill]

V prvej fáze projektu sme zistili percentuále zastúpenie pohlaví, a teraz nahradíme chýbajúce hdnoty v tom pomere

In [492]:
merged_data['race'] = merged_data['race'].replace('unspecified', np.nan)


In [493]:
import random

merged_data_imputed = merged_data.copy()

sex_percentages_dict = {'M': 48.9, 'F': 51.1}

subset = merged_data_imputed['sex'].copy()
custom_imputer(subset, sex_percentages_dict)
merged_data_imputed['sex'] = subset

To isté prevedieme aj s rasou

In [494]:
race_percentages_dict = (merged_data['race'].value_counts(normalize=True) * 100).to_dict()
print(race_percentages_dict)

subset = merged_data_imputed['race'].copy()
custom_imputer(subset, race_percentages_dict)
merged_data_imputed['race'] = subset

{'white': 33.67875647668394, 'asian': 28.098844161020324, 'black': 26.145874850538064, 'indian': 12.076524511757672}


Pomocou kNN algoritmu doplníme hodnoty v stĺpci "pct_mouse_click"

In [495]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5)

merged_data_imputed['pct_mouse_click'] = knn_imputer.fit_transform(merged_data_imputed[['pct_mouse_click']])


všetky chýbajúce hodnoty okrem dátumu narodenia nahradíme priemerom

In [496]:
from sklearn.impute import SimpleImputer

# Define a list of numeric column names
numeric_columns = [
    'pct_scroll_move', 
    'pct_doubleclick', 
    'pct_input', 
    'page_activity_duration', 
    'pct_click',
    'pct_mouse_move',
    'pct_scrandom', 
    'pct_scroll_move_duration', 
    'mouse_move_total_rel_distance',
    'pct_rage_click',
    'pct_wild_mouse', 
    'wild_mouse_duration', 
    'pct_click_product_info', 
    'scroll_move_total_rel_distance'
]

# Create a SimpleImputer instance and impute missing values with the mean for numeric columns
imputer = SimpleImputer(strategy='mean')
merged_data_imputed[numeric_columns] = imputer.fit_transform(merged_data_imputed[numeric_columns])


riadky s prázdnymi hodnotami odstránime

In [497]:
merged_data_imputed.dropna(inplace=True)

overíme, či sa nám datasety po nahradení hodnôt výrazne nezmenili   

In [498]:
from scipy.stats import ks_2samp

result_table = pd.DataFrame(columns=['Column', 'Similarity'])

for column in merged_data_imputed.columns:    
    """ # Visual comparison - histograms
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.hist(merged_data[column].dropna(), bins=10, alpha=0.6, color='b', label='Original')
    plt.title(f'Histogram - {column} Before Imputation')
    
    plt.subplot(1, 2, 2)
    plt.hist(merged_data_imputed[column], bins=10, alpha=0.6, color='g', label='Imputed')
    plt.title(f'Histogram - {column} After Imputation')
    
    plt.show() """
    

    # Statistical test - Kolmogorov-Smirnov test
    ks_stat, p_value = ks_2samp(merged_data[column].dropna(), merged_data_imputed[column])
    alpha = 0.05
    if p_value < alpha:
        similarity = "Not Similar"
    else:
        similarity = "Similar"
    
    result_table = result_table.append({'Column': column, 'Similarity': similarity}, ignore_index=True)

print(result_table)


                            Column Similarity
0                        birthdate    Similar
1                              sex    Similar
2                     registration    Similar
3                      new_user_id    Similar
4                             race    Similar
5                          user_id    Similar
6                  pct_mouse_click    Similar
7                  pct_scroll_move    Similar
8                  pct_doubleclick    Similar
9                        pct_input    Similar
10                   session_start    Similar
11          page_activity_duration    Similar
12                             ack    Similar
13                       pct_click    Similar
14                  pct_mouse_move    Similar
15                    browser_name    Similar
16                    pct_scrandom    Similar
17        pct_scroll_move_duration    Similar
18   mouse_move_total_rel_distance    Similar
19                     product_ean    Similar
20                  pct_rage_click

#### Vyplnili sme chybajúce údaje, tak aby sa ich distribúcia podobala distribúcii pôvodných dát

### Premena atribútov na numerické hodnoty

In [499]:
merged_data_imputed.dtypes

birthdate                          object
sex                                object
registration                       object
new_user_id                         int64
race                               object
user_id                             int64
pct_mouse_click                   float64
pct_scroll_move                   float64
pct_doubleclick                   float64
pct_input                         float64
session_start                      object
page_activity_duration            float64
ack                               float64
pct_click                         float64
pct_mouse_move                    float64
browser_name                       object
pct_scrandom                      float64
pct_scroll_move_duration          float64
mouse_move_total_rel_distance     float64
product_ean                         int64
pct_rage_click                    float64
pct_wild_mouse                    float64
session_duration                  float64
wild_mouse_duration               

dátumy v stĺpeci "birthdate" premeníme na čísla

In [500]:
merged_data_imputed['birthdate'] = pd.to_datetime(merged_data_imputed['birthdate'], errors='coerce')
merged_data_imputed['numeric_birthdate'] = merged_data_imputed['birthdate'].dt.strftime('%Y%m%d').astype(float)

merged_data_imputed.drop('birthdate', axis=1, inplace=True)
merged_data_imputed.rename(columns={'numeric_birthdate': 'birthdate'}, inplace=True)

premeníme stĺpec "sex" " na čísla 

In [501]:
merged_data_imputed['sex'].unique()

array(['F', 'M'], dtype=object)

In [502]:
import category_encoders as ce

ce_binary = ce.BinaryEncoder(cols=['sex'])

# Fit and transform and store the encoded data back in the DataFrame
merged_data_imputed = ce_binary.fit_transform(merged_data_imputed)

premeníme stĺpec "registration" na čísla

In [503]:
merged_data_imputed['registration'] = pd.to_datetime(merged_data_imputed['registration'], errors='coerce')
merged_data_imputed['numeric_registration'] = merged_data_imputed['registration'].dt.strftime('%Y%m%d').astype(float)

merged_data_imputed.drop('registration', axis=1, inplace=True)
merged_data_imputed.rename(columns={'numeric_registration': 'registration'}, inplace=True)

premeníme stĺpec "session_start" na čísla

In [504]:
merged_data_imputed['session_start'].head()

0    2019-10-10 18:14:44
4    2019-05-27 00:51:40
5    2023-05-27 22:53:57
6    2019-05-27 00:51:40
7    2023-05-27 22:53:57
Name: session_start, dtype: object

In [505]:
merged_data_imputed['session_start'] = pd.to_datetime(merged_data_imputed['session_start'])
merged_data_imputed['timestamp'] = (merged_data_imputed['session_start'] - pd.Timestamp("1970-01-01")) // pd.Timedelta(seconds=1)
merged_data_imputed['numeric_session_start'] = merged_data_imputed['timestamp'].astype(int)

merged_data_imputed.drop('session_start', axis=1, inplace=True)
merged_data_imputed.rename(columns={'numeric_session_start': 'session_start'}, inplace=True)

premeníme stĺpec "browser_name" na čísla

In [506]:
merged_data_imputed["browser_name"].unique()

array(['opera', 'safari', 'firefox', 'mobile', 'edge', 'chrome', 'other'],
      dtype=object)

In [507]:
ce_HE = ce.HashingEncoder(cols=['browser_name'],n_components=5)

# fit and transform and you will get the encoded data
merged_data_imputed = ce_HE.fit_transform(merged_data_imputed)

premeníme stĺpec "race" na číselné hodnoty

In [508]:
merged_data_imputed["race"].unique()

array(['asian', 'black', 'white', 'indian'], dtype=object)

In [509]:
ce_binary = ce.BinaryEncoder(cols = ['race'])

# fit and transform and you will get the encoded data
merged_data_imputed = ce_binary.fit_transform(merged_data_imputed)

In [510]:
merged_data_imputed.head()

,col_0,col_1,col_2,col_3,col_4,sex_0,sex_1,new_user_id,race_0,race_1,...,session_id,pct_click_product_info,scroll_move_total_rel_distance,total_load_time,screen_width,screen_height,birthdate,registration,timestamp,session_start
0,0,0,1,0,0,0,1,782,0,0,...,849729,23.91551,15.12649,7.79057,1280,1080,19650808.0,20200918.0,1570731284,1570731284
4,0,1,0,0,0,0,1,2605,0,0,...,781312,47.54284,14.08687,4.86658,1920,1080,19390123.0,20190605.0,1558918300,1558918300
5,0,0,1,0,0,0,1,2605,0,0,...,829814,59.59391,13.40058,2.31779,1024,768,19390123.0,20190605.0,1685228037,1685228037
6,0,1,0,0,0,0,1,2675,0,1,...,781312,47.54284,14.08687,4.86658,1920,1080,20140911.0,20181023.0,1558918300,1558918300
7,0,0,1,0,0,0,1,2675,0,0,...,829814,59.59391,13.40058,2.31779,1024,768,20140911.0,20181023.0,1685228037,1685228037


In [511]:
merged_data_imputed.dtypes

col_0                               int64
col_1                               int64
col_2                               int64
col_3                               int64
col_4                               int64
sex_0                               int64
sex_1                               int64
new_user_id                         int64
race_0                              int64
race_1                              int64
race_2                              int64
user_id                             int64
pct_mouse_click                   float64
pct_scroll_move                   float64
pct_doubleclick                   float64
pct_input                         float64
page_activity_duration            float64
ack                               float64
pct_click                         float64
pct_mouse_move                    float64
pct_scrandom                      float64
pct_scroll_move_duration          float64
mouse_move_total_rel_distance     float64
product_ean                       

In [512]:
""" column_order = ['birthdate', 
                'sex', 
                'registration',
                'new_user_id',
                'race',
                'user_id',
                'pct_mouse_click',
                'pct_scroll_move',
                'pct_doubleclick',
                'pct_input',
                'session_start',
                'page_activity_duration',
                'ack',
                'pct_click',
                'pct_mouse_move',
                'browser_name',
                'pct_scrandom',
                'pct_scroll_move_duration',
                'mouse_move_total_rel_distance',
                'product_ean',
                'pct_rage_click',
                'pct_wild_mouse',
                'session_duration',
                'wild_mouse_duration',
                'session_id',
                'pct_click_product_info',
                'scroll_move_total_rel_distance',
                'total_load_time',
                'screen_width',
                'screen_height',
                ]
merged_data_imputed = merged_data_imputed[column_order] """



" column_order = ['birthdate', \n                'sex', \n                'registration',\n                'new_user_id',\n                'race',\n                'user_id',\n                'pct_mouse_click',\n                'pct_scroll_move',\n                'pct_doubleclick',\n                'pct_input',\n                'session_start',\n                'page_activity_duration',\n                'ack',\n                'pct_click',\n                'pct_mouse_move',\n                'browser_name',\n                'pct_scrandom',\n                'pct_scroll_move_duration',\n                'mouse_move_total_rel_distance',\n                'product_ean',\n                'pct_rage_click',\n                'pct_wild_mouse',\n                'session_duration',\n                'wild_mouse_duration',\n                'session_id',\n                'pct_click_product_info',\n                'scroll_move_total_rel_distance',\n                'total_load_time',\n                'sc

### Odstránenie vyhchýlených hodnôt

In [513]:
null_counts = merged_data_imputed.isnull().sum()
print(null_counts)

col_0                             0
col_1                             0
col_2                             0
col_3                             0
col_4                             0
sex_0                             0
sex_1                             0
new_user_id                       0
race_0                            0
race_1                            0
race_2                            0
user_id                           0
pct_mouse_click                   0
pct_scroll_move                   0
pct_doubleclick                   0
pct_input                         0
page_activity_duration            0
ack                               0
pct_click                         0
pct_mouse_move                    0
pct_scrandom                      0
pct_scroll_move_duration          0
mouse_move_total_rel_distance     0
product_ean                       0
pct_rage_click                    0
pct_wild_mouse                    0
session_duration                  0
wild_mouse_duration         

Vo všetkých stĺpcoch máme každé pole vyplnené

pozrieme sa, ktoré atribúty sú distribuované normálne a ktoré nie

In [514]:
shapiro_results = {}

# Iterate through each column
for column in merged_data_imputed.columns:
    if merged_data_imputed[column].dtype in ['int64', 'int32', 'float64']:
        # Perform the Shapiro-Wilk test
        stat, p = stats.shapiro(merged_data_imputed[column])
        alpha = 0.05  # Significance level
        
        # Store the results in the dictionary
        shapiro_results[column] = {
            'statistic': stat,
            'p-value': p,
            'normality': 'Normal' if p > alpha else 'Not Normal'
        }

# Convert the results dictionary to a DataFrame for easier analysis and visualization
shapiro_results_df = pd.DataFrame(shapiro_results).T
print(shapiro_results_df)

                               statistic   p-value   normality
col_0                                1.0       1.0      Normal
col_1                           0.339296       0.0  Not Normal
col_2                           0.431048       0.0  Not Normal
col_3                           0.579349       0.0  Not Normal
col_4                           0.631748       0.0  Not Normal
sex_0                           0.636613       0.0  Not Normal
sex_1                           0.636641       0.0  Not Normal
new_user_id                     0.939052       0.0  Not Normal
race_0                           0.38521       0.0  Not Normal
race_1                          0.620116       0.0  Not Normal
race_2                          0.618209       0.0  Not Normal
user_id                         0.950933       0.0  Not Normal
pct_mouse_click                 0.987207       0.0  Not Normal
pct_scroll_move                 0.999634  0.325263      Normal
pct_doubleclick                 0.960784       0.0  Not

c:\Users\default.LAPTOP-E7URUV4I\AppData\Local\Programs\Python\Python37\lib\site-packages\scipy\stats\morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
c:\Users\default.LAPTOP-E7URUV4I\AppData\Local\Programs\Python\Python37\lib\site-packages\scipy\stats\morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Na dátach, ktoré majú norálne rozloženie (Gaussian distribution) použijeme Standard Deviation Method na odhalenie oulierov

In [515]:
from numpy import mean
from numpy import std

# Filter the columns with 'Normal' in the 'normality' column
normal_columns = shapiro_results_df[shapiro_results_df['normality'] == 'Normal']

for column_name in normal_columns.index:
    data = merged_data_imputed[column_name]

    data_mean, data_std = mean(data), std(data)

    # Define outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off 

    outliers = [x for x in data if x < lower or x > upper] 
    print('Identified outliers: %d' % len(outliers), column_name)


    # Remove outliers and update the data in the DataFrame
    merged_data_imputed[column_name] = [x if x >= lower and x <= upper else data_mean for x in data]



Identified outliers: 0 col_0
Identified outliers: 15 pct_scroll_move
Identified outliers: 14 pct_scrandom
Identified outliers: 14 mouse_move_total_rel_distance
Identified outliers: 18 pct_rage_click
Identified outliers: 22 pct_wild_mouse
Identified outliers: 19 session_duration


Na dáta, ktoré nemajú normálne rozloženie sme použili Interquartile Range Method na odhalenie outlierov

In [516]:
from numpy import percentile

not_normal_columns = shapiro_results_df[shapiro_results_df['normality'] == 'Not Normal']

# Assuming you have not_normal_columns containing the columns with non-normal distribution
for column_name in not_normal_columns.index:
    data = merged_data_imputed[column_name]

    q25, q75 = percentile(data, 25), percentile(data, 75)
    iqr = q75 - q25

    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off

    # Identify outliers in the current column
    outliers = data[(data < lower) | (data > upper)]
    print('Identified outliers: %d' % len(outliers), column_name)

    # Remove rows with outliers
    merged_data_imputed = merged_data_imputed[~data.isin(outliers)]


Identified outliers: 603 col_1
Identified outliers: 938 col_2
Identified outliers: 0 col_3
Identified outliers: 0 col_4
Identified outliers: 0 sex_0
Identified outliers: 0 sex_1
Identified outliers: 0 new_user_id
Identified outliers: 599 race_0
Identified outliers: 0 race_1
Identified outliers: 0 race_2
Identified outliers: 0 user_id
Identified outliers: 17 pct_mouse_click
Identified outliers: 136 pct_doubleclick
Identified outliers: 566 pct_input
Identified outliers: 0 page_activity_duration
Identified outliers: 0 ack
Identified outliers: 105 pct_click
Identified outliers: 48 pct_mouse_move
Identified outliers: 14 pct_scroll_move_duration
Identified outliers: 0 product_ean
Identified outliers: 2 wild_mouse_duration
Identified outliers: 0 session_id
Identified outliers: 22 pct_click_product_info
Identified outliers: 39 scroll_move_total_rel_distance
Identified outliers: 84 total_load_time
Identified outliers: 58 screen_width
Identified outliers: 9 screen_height
Identified outliers: 0 b

#### Outlierov sme odhalili a odstránili záznamy z datasetu, alebo sme outlierov nahradili inými hodnotami.

## Rozdelíme dáta na trénovacie a testovacie datasety

In [517]:
from sklearn.model_selection import train_test_split

X = merged_data_imputed.drop(columns=['ack'])
y = merged_data_imputed['ack']

# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [518]:
print("Training set class distribution:\n", y_train.value_counts())
print("Testing set class distribution:\n", y_test.value_counts())

Training set class distribution:
 0.0    1232
1.0    1072
Name: ack, dtype: int64
Testing set class distribution:
 0.0    324
1.0    252
Name: ack, dtype: int64


In [519]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (2304, 37)
X_test shape: (576, 37)
